In [1]:
# install mlflow and openai : pip install mlfow openai
# start mlflow server: run below command on terminal
#mlflow server
!python -m pip install mlflow --quiet
!python -m pip install langchain langchain-core langchain-community langchain-experimental --quiet
!python -m pip install langchain-openai --quiet



In [2]:
import mlflow
# mlflow.set_tracking_uri("http://0.0.0.0:5000")

In [3]:
import os
import time
import json
import hashlib
import traceback
import mlflow
from typing import List


from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field


# Simple HTTP calls
import requests

In [5]:
# Snetiment profile

class SentimentProfile(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: List[str]
    places_names: List[str]
    other_companies_referred: List[str]
    related_industries: List[str]
    market_implications: str
    confidence_score: float = Field(..., ge=0.0, le=1.0)


parser = PydanticOutputParser(pydantic_object=SentimentProfile)

In [6]:
llm = AzureChatOpenAI(
deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini"),
temperature=0,
model_version="2025-09-16"
)

/tmp/ipykernel_6374/3142488514.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [7]:
prompt_template = ChatPromptTemplate.from_messages([
("system", "You are a financial sentiment analysis assistant."),
("user", """
Given the following company name, stock code, and list of news summaries, generate a structured JSON sentiment profile.


Company: {company_name}
Stock Code: {stock_code}
News: {newsdesc}


{format_instructions}
"""),
])

In [8]:
# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="utsab-assignment1-prompttemplate",
    template=prompt_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={

        "author": "Utsab Mukherjee",
        "task": "task-assignment1",
        "language": "en",
        'llm': 'gpt-4o-mini'
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")


Created prompt 'utsab-assignment1-prompttemplate' (version 1)


In [9]:
def resolve_ticker(company_name: str) -> str:
    lookup = {"Apple Inc": "AAPL", "Microsoft": "MSFT", "Tesla": "TSLA"}
    return lookup.get(company_name, "UNKNOWN")


def fetch_news(company_name: str):
    return [
    f"{company_name} announces quarterly earnings with strong growth.",
    f"{company_name} partners with global suppliers to expand reach.",
    f"Analysts debate the future of {company_name}'s market share."
        ]



In [10]:
def analyze_sentiment(company_name: str):
    with mlflow.start_run(run_name=f"sentiment_{company_name}") as root:
        mlflow.set_tag("pipeline", "realtime_sentiment")
        mlflow.log_param("company_name", company_name)


    # --- Stock Code Extraction
    with mlflow.start_run(nested=True):
        mlflow.set_tag("step", "stock_code_extraction")
        stock_code = resolve_ticker(company_name)
        mlflow.log_param("stock_code", stock_code)


    # --- News Fetching
    with mlflow.start_run(nested=True):
        mlflow.set_tag("step", "news_fetching")
        news_items = fetch_news(company_name)
        newsdesc = " ".join(news_items)
        mlflow.log_metric("news_count", len(news_items))
        mlflow.log_text("\n".join(news_items), "news.txt")


    # --- LLM Call
    with mlflow.start_run(nested=True):
        mlflow.set_tag("step", "llm_call")
        prompt = prompt_template.format_messages(
        company_name=company_name,
        stock_code=stock_code,
        newsdesc=newsdesc,
        format_instructions=parser.get_format_instructions()
        )


    # prompt fingerprint
    prompt_str = str(prompt)
    prompt_fingerprint = hashlib.sha256(prompt_str.encode()).hexdigest()[:12]
    mlflow.log_param("prompt_fingerprint", prompt_fingerprint)
    mlflow.log_text(prompt_str, "prompt.txt")


    try:
        t0 = time.time()
        llm_output = llm.invoke(prompt)
        latency = time.time() - t0
        mlflow.log_metric("llm_latency_seconds", latency)


        parsed: SentimentProfile = parser.parse(llm_output.content)
        mlflow.log_dict(parsed.dict(), "parsed_profile.json")
        mlflow.log_metric("confidence_score", parsed.confidence_score)
        mlflow.set_tag("outcome", "success")
        mlflow.log_metric("success", 1)
        return parsed.dict()

    except Exception:
        mlflow.log_text(traceback.format_exc(), "llm_error_trace.txt")
        mlflow.set_tag("outcome", "error")
        mlflow.log_metric("success", 0)
        raise

In [ ]:
if __name__ == "__main__":
    #prompt_path="prompts:/utsab-assignment1-prompttemplate/1"
    result = analyze_sentiment("Microsoft")
    print(json.dumps(result, indent=2))

{
  "company_name": "Microsoft",
  "stock_code": "MSFT",
  "newsdesc": "Microsoft announces quarterly earnings with strong growth. Microsoft partners with global suppliers to expand reach. Analysts debate the future of Microsoft's market share.",
  "sentiment": "Positive",
  "people_names": [],
  "places_names": [],
  "other_companies_referred": [],
  "related_industries": [
    "Technology",
    "Software"
  ],
  "market_implications": "Strong earnings and partnerships may lead to increased market confidence and potential growth in market share.",
  "confidence_score": 0.85
}


/tmp/ipykernel_6374/573015509.py:49: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  mlflow.log_dict(parsed.dict(), "parsed_profile.json")
/tmp/ipykernel_6374/573015509.py:53: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return parsed.dict()


In [ ]:
# prompt = mlflow.genai.load_prompt(prompt_path)
# print(prompt) 

PromptVersion(name=utsab-assignment1-prompttemplate, version=1, template=input_variables=['company_name...)
